# Light curve

In this notebook we'll assemble the light curve of a star, calibrated against other stars of the same field, with the purpose of uncovering an exoplanet transit.

As usual, you can download this page as a {download}`jupyter notebook <./lightcurve.ipynb>` file.

```{note}
In case you have been working on an AIfA lab room computer until this point, you might be interested in moving to a personal laptop (for example) for these last steps.
All that we'll need from here on is the content of the `PHOTOMETRY` directory (including the reference catalog), as well as **one** of the pre-reduced images (from `LIGHT_PRERED`). This is a relatively small amount of data (compared to the raw or pre-reduced images), so you could consider to copy just these files to your own computer. The CPU requirements for these last steps will also be negligible. 
```

In [ ]:
import dataredconfig
import pathlib

import numpy as np
import astropy
import astropy.table
import astropy.visualization
import astropy.coordinates
from astropy import units as u

import datetime

%matplotlib widget
import matplotlib
from matplotlib import pyplot as plt

import ccdproc

## Preparation of the data

In [ ]:
photometry_dir = dataredconfig.work_dir / "PHOTOMETRY"

object_to_process = "HD92670"

catalog_filepaths = sorted(list(photometry_dir.glob('*.fits')))
catalogs = []

# We'll read all catalogs in, can keep only those matching the desired object in the above list.
for catalog_filepath in catalog_filepaths:
    
    catalog = astropy.table.Table.read(catalog_filepath)

    # We select the photometric catalogs of our object:  
    if "OBJECT" in catalog.meta:
        if catalog.meta["OBJECT"] == object_to_process:
            print(f"{catalog_filepath} : {catalog.meta}")
            catalogs.append(catalog)


In [ ]:
# We combine these catalogs into a single table, in "depth": columns will be 2D, where the second dimension is time.
catalog = astropy.table.dstack(catalogs, join_type="exact", metadata_conflicts="silent")

# We also produce a list of datetime objects, from the FITS headers:
date_strings = [c.meta["DATE-OBS"] for c in catalogs]
dates = [datetime.datetime.fromisoformat(date) for date in date_strings]

# And while we are at it, same for the airmass:
airmasses = [c.meta["AIRMASS"] for c in catalogs]

# We read the reference catalog, as this one contains the position of each star
ref_catalog = astropy.table.Table.read(photometry_dir / f"ref_catalog_{object_to_process}.fits")
assert len(ref_catalog) == len(catalog) # Just a check that these are indeed of same length

# We copy the positions from the reference catalog over to our combined catalog:
catalog["sky_centroid_win"] = ref_catalog["sky_centroid_win"]

print(f"Number of epochs: {len(catalogs)}")

# And just for your information, the column names of our "combined" catalogue are:
catalog.colnames


We display an image of the field, overplotting the "source indices" corresponding to rows of our catalog.

This will allow us to identify the target and reference stars. 

In [ ]:
# We load one of the images, it does not have to be a specific one.
light_prered_dir = dataredconfig.work_dir / "LIGHT_PRERED"
science_files = ccdproc.ImageFileCollection(light_prered_dir, keywords=dataredconfig.ifc_header_keywords)
science_files = science_files.filter(object=object_to_process)
image_path = science_files.files[0]
image = ccdproc.CCDData.read(image_path, unit="adu")
image.data -= np.median(image.data) # Quick sky subtraction

# And now create the figure
plt.figure(figsize=(10, 6))
ax = plt.subplot(projection=image.wcs)
ax.imshow(image.data, origin='lower', cmap='Greys_r', interpolation='nearest',
    norm=astropy.visualization.simple_norm(image.data, stretch="sqrt", min_cut=-20, max_cut=500))
ax.scatter(
    catalog["sky_centroid_win"].ra.degree,
    catalog["sky_centroid_win"].dec.degree,
    transform=ax.get_transform('world'),
    s=50, # The size of these markers is not related to any measurement apertures!
    edgecolor='red', facecolor='none'
    )
for line in catalog:
    ax.text(
        x=line["sky_centroid_win"].ra.degree,
        y=line["sky_centroid_win"].dec.degree,
        s=str(line.index),
        transform=ax.get_transform('world'),
        color="cyan"
        )
ax.grid(color='white', ls='solid')
ax.coords[0].set_axislabel('RA')
ax.coords[1].set_axislabel('Dec')
#ax.coords[0].set_ticks(spacing=5.*u.arcmin)
#ax.coords[1].set_ticks(spacing=5.*u.arcmin)
plt.tight_layout()
plt.show()



Let's now visualize the raw light curves, to get a first impression.
The following code is there as an example on how to use and manipulate the `catalog` and how to plot basic light curves, to get you started.

In [ ]:

# We compute an "instrumental magnitude" from all our flux measurements.
# "Instrumental" means that this is not yet calibrated to correspond to an apparent magnitude.
# Note that this is the line where we select which aperture to consider.
# Of course, you can try to figure out which apertures works best for your data.
catalog["instr_mag"] = -2.5 * np.log10(catalog["sum_8"].value) # this is a "2D" column: (source index, date)

# We can also compute summary statistics for each source. Useful ones could be for example the median instrumental magnitude,
# obtained by taking the median along the "date"-axis of the instrumental magnitudes:
catalog["mean_instr_mag"] = np.nanmedian(catalog["instr_mag"].value, axis=1) # this is just a 1D column: (source index)
# Using "nanmedian" instead of "median" has the advantage that NaNs get ignored.
# And similarly the standard deviation of each light curve:
catalog["std_instr_mag"] = np.nanstd(catalog["instr_mag"].value, axis=1) # this is just a 1D column: (source index)
# Note that these are 1D columns: they just have one index, namely the source index.

# Another interesting quantity might be the angular separation (in degrees) between each star and your target:
target_index = 34 # Adapt this!
target_center_pos = catalog["sky_centroid_win"][target_index]
catalog["separation"] = astropy.coordinates.SkyCoord.separation(catalog["sky_centroid_win"], target_center_pos)
# Depending on your data, it could be important to avoid stars with large angular separation to calibrate your target.

# When making a plot (and also to select reference stars), you could hand-pick indices, or build such a list algorithmically:
indices_show = [0, 5, 8, 17, 38]

plt.figure()
ax = plt.subplot()

# We simply loop over the indices to show:
for index in indices_show:
    ax.plot(dates, catalog["instr_mag"][index], lw=1, label=index)

ax.invert_yaxis() # Needed, as we show a magnitude on y.

# Some advanced settings to help getting a nice format of the date axis labels:
ax.xaxis.set_major_formatter(matplotlib.dates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.set_xlabel("UTC")
ax.set_ylabel("Instrumental magnitude")
plt.gcf().autofmt_xdate()
plt.legend()
plt.tight_layout()
plt.show()

It's good to play around with these raw light curves a bit.

```{admonition} Question
Do you observe any trends in these light curves, common to all stars or to some of these stars? Comment on what could cause these trends.
```

```{note} 
An interesting plot could be a scatter plot of the mean (instrumental) magnitude versus the standard deviation of each light curve (i.e., source). This could help you identify (for example) a magnitude threshold beyond which stars are saturated, and therefore can't be used for calibration. Indeed, stars that are close to saturation will typically show an excess scatter in their light curves, compared to stars that are safely below saturation.
```


## Calibration

To reveal the transit itself, some empirical calibration of the flux (or magnitude) measured in each exposure is needed.

```{admonition} Question
In theory, what would be the properties of good reference stars to use for the calibration?
```

It's good to keep in mind that some technical or instrumental effects might possibly have strong effects on the calibration, and influence the optimal selection of reference stars. So in practice, it's mandatory to actually test the calibration, instead of relying on theoretical considerations.

Before implementing a calibration, we now need to define a strategy.

```{admonition} Question
What causes the flux variations between exposures and what is the best way to correct for them? Try to come up with a formalism (i.e., equations) of a simple correction strategy.
```





## Plotting the light curves

Once we have computed a calibration, we can plot the light curve of the exoplanet system!


```{admonition} Question
In addition to the light curve of the target star, also plot the light curves of some reference stars (after calibration), on the same axes or a separate figure. Use this to verify that your corrections are successful, and that you did not pick up variable stars or stars that have observational issues as reference sources.
Do you see residual correlations between some stars?
```


```{admonition} Question
If you see stars with strong residuals after the calibration, what could be the reasons?
You can try to optimize the calibration by adapting the code to ignore particular stars.
```

```{admonition} Question
Can you identify the start and/or end of the transit?
Give an estimate for the uncertainty of your estimate.
```





## Discussion of the exoplanet transit

```{admonition} Question
What "depth" (in magnitude) do you measure on your transit. Can you convert your value to the commonly used "parts per thousand" (ppt)?
```

```{admonition} Question
Are your findings regarding the transit consistent with "literature" measurements (depth, duration, start/end time of the transit)?
```

```{admonition} Question
Calculate the radius of the exoplanet relative to its host star from your measurement of the light curve. Think as well of how you could estimate the uncertainties.
```